In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from PIL import Image
from sklearn.metrics import average_precision_score
from torch.utils.data import DataLoader, TensorDataset, random_split
import numpy as np

In [17]:
# Paths
dataset_path = '/content/drive/MyDrive/CUB_200_2011/CUB_200_2011/images'
output_path = '/content/drive/MyDrive/CUB_200_2011/limited_data'
os.makedirs(output_path, exist_ok=True)

# Preprocessing pipeline
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [18]:
# Preprocess a maximum of 1,000 images
all_images = []
all_labels = []
max_images = 1000
image_counter = 0

for class_folder in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_folder)
    if os.path.isdir(class_path):
        label = int(class_folder.split('.')[0]) - 1  # Convert to 0-based index
        images = os.listdir(class_path)
        for img_file in images:
            img_path = os.path.join(class_path, img_file)
            try:
                img = Image.open(img_path).convert("RGB")
                img_tensor = transform(img)
                all_images.append(img_tensor)
                all_labels.append(label)
                image_counter += 1
                if image_counter >= max_images:
                    break
            except Exception as e:
                print(f"Error processing {img_path}: {e}")
        if image_counter >= max_images:
            break

# Save preprocessed data
all_images = torch.stack(all_images)
all_labels = torch.tensor(all_labels)
torch.save({'images': all_images, 'labels': all_labels}, os.path.join(output_path, 'limited_data.pt'))

print(f"Dataset preprocessed. Total images: {len(all_images)}. Saved at {output_path}.")


Dataset preprocessed. Total images: 1000. Saved at /content/drive/MyDrive/CUB_200_2011/limited_data.


In [19]:
# Load preprocessed data
data = torch.load('/content/drive/MyDrive/CUB_200_2011/limited_data/limited_data.pt')
images, labels = data['images'], data['labels']

# Convert labels to one-hot encoding
def to_one_hot(labels, num_classes):
    return torch.eye(num_classes)[labels]

num_classes = 200  # Total number of classes
labels_one_hot = to_one_hot(labels, num_classes)

# Create TensorDataset
dataset = TensorDataset(images, labels_one_hot)

# Split into train and test sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

print(f"Dataset split: {train_size} training samples, {test_size} testing samples.")

<ipython-input-19-d1d4e9c683bc>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load('/content/drive/MyDrive/CUB_200_2011/limited_data/limited_data.pt')


Dataset split: 800 training samples, 200 testing samples.


In [20]:
# Define the multi-label classification model
class MultiLabelCNN(nn.Module):
    def __init__(self, num_classes):
        super(MultiLabelCNN, self).__init__()
        self.backbone = models.resnet50(pretrained=True)
        self.backbone.fc = nn.Sequential(
            nn.Linear(self.backbone.fc.in_features, num_classes),
            nn.Sigmoid()  # Sigmoid for multi-label output
        )

    def forward(self, x):
        return self.backbone(x)

# Initialize the model
model = MultiLabelCNN(num_classes)
print("Model initialized.")

# Initialize label similarity graph (identity matrix)
label_similarity_graph = np.eye(num_classes)
label_similarity_graph = torch.tensor(label_similarity_graph, dtype=torch.float32)

print("Initialized label similarity graph (identity matrix).")

# Collaborative loss function
def collaborative_loss(predictions, targets, similarity_graph, lambda_graph=0.01):
    bce_loss = nn.BCELoss()(predictions, targets)
    graph_term = torch.sum(similarity_graph * torch.mm(predictions.T, predictions)) / predictions.size(0)
    return bce_loss + lambda_graph * graph_term

# Refine the similarity graph
def refine_similarity_graph(predictions, current_graph, alpha=0.9):
    co_occurrence = torch.mm(predictions.T, predictions) / predictions.size(0)
    updated_graph = alpha * current_graph + (1 - alpha) * co_occurrence
    return updated_graph

# Training loop
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10
model.train()
for epoch in range(num_epochs):
    epoch_loss = 0
    for imgs, lbls in train_loader:
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = collaborative_loss(outputs, lbls.float(), label_similarity_graph)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/len(train_loader):.4f}")

    # Refine the similarity graph
    with torch.no_grad():
        all_predictions = []
        for imgs, _ in train_loader:
            outputs = model(imgs)
            all_predictions.append(outputs)
        all_predictions = torch.cat(all_predictions)
        label_similarity_graph = refine_similarity_graph(all_predictions, label_similarity_graph)

    print(f"Label similarity graph refined at epoch {epoch+1}.")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model initialized.
Initialized label similarity graph (identity matrix).
Epoch [1/10], Loss: 0.0625
Label similarity graph refined at epoch 1.
Epoch [2/10], Loss: 0.0157
Label similarity graph refined at epoch 2.
Epoch [3/10], Loss: 0.0135
Label similarity graph refined at epoch 3.
Epoch [4/10], Loss: 0.0123
Label similarity graph refined at epoch 4.
Epoch [5/10], Loss: 0.0102
Label similarity graph refined at epoch 5.
Epoch [6/10], Loss: 0.0096
Label similarity graph refined at epoch 6.
Epoch [7/10], Loss: 0.0078
Label similarity graph refined at epoch 7.
Epoch [8/10], Loss: 0.0071
Label similarity graph refined at epoch 8.
Epoch [9/10], Loss: 0.0068
Label similarity graph refined at epoch 9.
Epoch [10/10], Loss: 0.0067
Label similarity graph refined at epoch 10.


In [21]:
from sklearn.metrics import average_precision_score
import numpy as np

def evaluate_model(model, test_loader):
    """
    Evaluates the model using Mean Average Precision (mAP).
    Filters out classes with no positive samples in the ground truth.
    """
    model.eval()
    all_labels = []
    all_predictions = []

    # Collect all predictions and ground truth labels
    with torch.no_grad():
        for imgs, lbls in test_loader:
            outputs = model(imgs)  # Model predictions
            all_predictions.append(outputs.numpy())
            all_labels.append(lbls.numpy())

    # Convert to numpy arrays
    all_predictions = np.concatenate(all_predictions)
    all_labels = np.concatenate(all_labels)

    # Identify valid classes (at least one positive sample in y_true)
    valid_classes = (all_labels.sum(axis=0) > 0).nonzero()[0]
    if len(valid_classes) == 0:
        print("No valid classes with positive samples found.")
        return 0.0

    # Filter valid classes
    all_labels_filtered = all_labels[:, valid_classes]
    all_predictions_filtered = all_predictions[:, valid_classes]

    # Compute mAP
    mAP = average_precision_score(all_labels_filtered, all_predictions_filtered, average="macro")
    print(f"Valid Classes: {len(valid_classes)} / {all_labels.shape[1]}")
    print(f"Mean Average Precision (mAP): {mAP:.4f}")
    return mAP

# Evaluate the model
mAP_score = evaluate_model(model, test_loader)

Valid Classes: 18 / 200
Mean Average Precision (mAP): 0.8214
